In [2]:
import ffmpeg
import subprocess

def get_duration(filename):
    """Devuelve la duración en segundos usando ffprobe."""
    result = subprocess.run(
        ['ffprobe', '-v', 'error', '-show_entries', 'format=duration',
         '-of', 'default=noprint_wrappers=1:nokey=1', filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

def reduce_video_size(input_video, output_video, target_size_mb):
    duration = get_duration(input_video)
    # Tamaño deseado en bits
    target_size_bits = target_size_mb * 8 * 1024 * 1024
    # Reserva un poco para el audio (~128 kbps)
    audio_bitrate = 128 * 1024
    # Bitrate total objetivo (en bits por segundo)
    total_bitrate = int((target_size_bits / duration) - audio_bitrate)
    if total_bitrate < 100_000:
        raise ValueError("El tamaño objetivo es demasiado pequeño para mantener una calidad aceptable.")

    print(f"Duración: {duration:.1f} segundos")
    print(f"Bitrate de video: {total_bitrate // 1000} kbps")
    print(f"Bitrate de audio: {audio_bitrate // 1000} kbps")

    ffmpeg.input(input_video)\
        .output(output_video,
                vcodec='libx264',
                video_bitrate=total_bitrate,
                acodec='copy',
                audio_bitrate='128k',
                preset='veryfast')\
        .run(overwrite_output=True)

    print(f"¡Video comprimido a aproximadamente {target_size_mb} MB!")

# --- USO ---
reduce_video_size('video.mp4', 'video_reducido.mp4', target_size_mb=15.5)


Duración: 33.8 segundos
Bitrate de video: 3711 kbps
Bitrate de audio: 131 kbps


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

¡Video comprimido a aproximadamente 15.5 MB!


[out#0/mp4 @ 0x55aded5d81c0] video:13011kB audio:1310kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.376617%
frame= 2028 fps=226 q=-1.0 Lsize=   14375kB time=00:00:33.81 bitrate=3482.7kbits/s speed=3.77x    
[libx264 @ 0x55aded637540] frame I:18    Avg QP:20.20  size: 99949
[libx264 @ 0x55aded637540] frame P:549   Avg QP:22.70  size: 13821
[libx264 @ 0x55aded637540] frame B:1461  Avg QP:23.59  size:  2694
[libx264 @ 0x55aded637540] consecutive B-frames:  3.3%  1.5%  1.8% 93.5%
[libx264 @ 0x55aded637540] mb I  I16..4: 37.0% 37.3% 25.8%
[libx264 @ 0x55aded637540] mb P  I16..4:  6.1%  4.8%  0.7%  P16..4: 16.2%  5.5%  2.4%  0.0%  0.0%    skip:64.3%
[libx264 @ 0x55aded637540] mb B  I16..4:  0.7%  0.3%  0.0%  B16..8:  3.8%  1.2%  0.2%  direct: 4.1%  skip:89.7%  L0:34.3% L1:50.2% BI:15.5%
[libx264 @ 0x55aded637540] final ratefactor: 20.75
[libx264 @ 0x55aded637540] 8x8 transform intra:39.2% inter:33.5%
[libx264 @ 0x55aded637540] coded y,uvDC,uvAC intra: 30.2% 40.6% 14.